In [23]:
import pandas as pd
import numpy as np

## Nth Highest Salary

In [24]:
employee_dict = {
    'id': [1,2,3],
    'salary': [100,200,300]
}
N = 2

employee = pd.DataFrame(employee_dict)
employee

,id,salary
0,1,100
1,2,200
2,3,300


Write a solution to find the nth highest salary from the Employee table. If there is no nth highest salary, return null.

In [25]:
col = 'getNthHighestSalary' + '(' + str(N) + ')'
unique_salaries = employee.drop_duplicates('salary')
if (len(unique_salaries)>=N) and (N>0):
    NthHighestSalary = employee.nlargest(N, 'salary')['salary'].iloc[-1]
    print( pd.DataFrame({col: [NthHighestSalary]}) )
else:
    print( pd.DataFrame({col: [np.NaN]}) )

   getNthHighestSalary(2)
0                     200


## Department Highest Salary

In [26]:
employee_dict = {
     'id' : [1,2,3,4,5], 
     'name' : ['Joe', 'Jim', 'Henry', 'Sam', 'Max'],
     'salary' : [70000, 90000, 80000, 60000, 90000],
     'departmentId' : [1,1,2,2,1] 
}

department_dict = {
    'id' : [1,2],
    'name' : ['IT', 'Sales']
}

employee = pd.DataFrame(employee_dict)
department = pd.DataFrame(department_dict)

print(employee, "\n", department)

   id   name  salary  departmentId
0   1    Joe   70000             1
1   2    Jim   90000             1
2   3  Henry   80000             2
3   4    Sam   60000             2
4   5    Max   90000             1 
    id   name
0   1     IT
1   2  Sales


Write a solution to find employees who have the highest salary in each of the departments.

In [27]:
# Merge employee and department data frames on department id and fetch necessary columns then rename them
merged = pd.merge(left=employee, right=department, how='inner', left_on='departmentId', right_on='id')[['name_y', 'name_x', 'salary']]

mapper = {'name_y':'Department', 'name_x':'Employee', 'salary':'Salary'}

merged.rename(mapper, axis=1, inplace=True)

merged

,Department,Employee,Salary
0,IT,Joe,70000
1,IT,Jim,90000
2,IT,Max,90000
3,Sales,Henry,80000
4,Sales,Sam,60000


In [28]:
# Find maximum salary per department by grouping the above merged data frame
MaxSalaryPerDepartment = merged.groupby(by='Department').max('Salary').rename(columns={'Salary':'MaxSalary'})

MaxSalaryPerDepartment

,MaxSalary
Department,
IT,90000
Sales,80000


In [29]:
# Merge the earlier data frame with MaxSalaryPerDepartment on deparment and filter the rows where Salary and MaxSalary are the same
MergedWithMax = pd.merge(merged, MaxSalaryPerDepartment, how='inner', on='Department')

MergedWithMax[MergedWithMax['Salary'] == MergedWithMax['MaxSalary']][['Department', 'Employee', 'Salary']]

,Department,Employee,Salary
1,IT,Jim,90000
2,IT,Max,90000
3,Sales,Henry,80000


In [30]:
result = merged.groupby(by='Department')['Salary'].transform('max')

result

0    90000
1    90000
2    90000
3    80000
4    80000
Name: Salary, dtype: int64

## Rank Scores

In [31]:
scores_dict = {
    'id': [1,2,3,4,5,6],
    'score': [3.5,3.65,4.0,3.85,4.0,3.65]
}

scores = pd.DataFrame(scores_dict)
scores

,id,score
0,1,3.50
1,2,3.65
2,3,4.00
3,4,3.85
4,5,4.00
5,6,3.65


Write a solution to find the rank of the scores. The ranking should be calculated according to the following rules:

The scores should be ranked from the highest to the lowest.
If there is a tie between two scores, both should have the same ranking.
After a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no holes between ranks.
Return the result table ordered by score in descending order.

In [32]:
# Sort the data frame scores by score descending and create a rank column using rank method.
scores['rank'] = scores['score'].rank(method='dense', ascending=False).astype('int')

# Return the required columns by sorting rank ascending
scores[['score','rank']].sort_values(by='rank')

,score,rank
2,4.00,1
4,4.00,1
3,3.85,2
1,3.65,3
5,3.65,3
0,3.50,4


In [33]:
# Create a new data frame with the existing columns and calculate the new column i.e rank
ranked_scores = scores.assign(rank=scores['score'].rank(method='dense', ascending=False).astype(int))

# Return the required columns by sorting rank ascending 
ranked_scores.sort_values(by='rank')[['score', 'rank']]

,score,rank
2,4.00,1
4,4.00,1
3,3.85,2
1,3.65,3
5,3.65,3
0,3.50,4


## Delete Duplicate Emails

In [35]:
person_dict = {
    'id': [1,2,3],
    'email': ['john@example.com', 'bob@example.com', 'john@example.com']
}

persons = pd.DataFrame(person_dict)
persons

,id,email
0,1,john@example.com
1,2,bob@example.com
2,3,john@example.com


Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.e.

For Pandas users, please note that you are supposed to modify Person in place.

After running your script, the answer shown is the Person table. The driver will first compile and run your piece of code and then show the Person table. The final order of the Person table does not matter.

In [43]:
persons.sort_values('id', inplace=True)
persons.drop_duplicates('email', inplace=True)

In [44]:
persons

,id,email
0,1,john@example.com
1,2,bob@example.com


## Rearrange Products Table

In [48]:
product_dict = {
    'product_id' : [0,1],
    'store1' : [95,70],
    'store2' : [100, np.NaN],
    'store3' : [105, 80]
}

product = pd.DataFrame(product_dict)
product

,product_id,store1,store2,store3
0,0,95,100.0,105
1,1,70,NaN,80


Write a solution to rearrange the Products table so that each row has (product_id, store, price). If a product is not available in a store, do not include a row with that product_id and store combination in the result table.

In [50]:
pd.melt(product, id_vars='product_id', value_vars=['store1', 'store2', 'store3'], var_name='store', value_name='price').dropna()

,product_id,store,price
0,0,store1,95.0
1,1,store1,70.0
2,0,store2,100.0
4,0,store3,105.0
5,1,store3,80.0
